## Lab 12-3

## Source - Short Sentense

In [1]:
import tensorflow as tf
import numpy as np

sample = " if you want you"
idx2char = list(set(sample)) # 유니크한 문자열을 리스트로 넣는다.
char2idx = {c: i for i, c in enumerate(idx2char)} # 리스트 문자열을 index로 매칭되는 딕셔너리를 만든다.

# hyper parameters
dic_size = len(char2idx)
rnn_hidden_size = len(char2idx)
num_classes = len(char2idx)
batch_size = 1
sequence_length = len(sample)-1

sample_idx = [char2idx[c] for c in sample] # char to index
x_data = [sample_idx[:-1]] # X data sample (0 ~ n-1) hello:hell
y_data = [sample_idx[1:]]  # Y label sample ( 1~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

X_one_hot = tf.one_hot(X, num_classes)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size,  tf.float32)
outputs, _state = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)

loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        _loss, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})
        
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        # '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost)
        print('{0:04d} loss: {1:.9f} prediction: {2}'.format(i, _loss, ''.join(result_str)))


0000 loss: 2.317929506 prediction: uuuuuuuuuuu uu 
0001 loss: 2.209185362 prediction: uuuuuuuuuuu uuu
0002 loss: 2.153294086 prediction: u  uu  uuu  uu 
0003 loss: 2.090982676 prediction: y  oo  ooo  oo 
0004 loss: 2.016361475 prediction: y  oo  ooo  you
0005 loss: 1.925761104 prediction: y  oou ooo  you
0006 loss: 1.827335000 prediction: y  you yoo  you
0007 loss: 1.745518088 prediction: y  you yoot you
0008 loss: 1.660204649 prediction: y  you yott you
0009 loss: 1.597371459 prediction: y  you yatt you
0010 loss: 1.535698295 prediction: y  you yaot you
0011 loss: 1.484851718 prediction: y  you waot you
0012 loss: 1.431192160 prediction: y  you watt you
0013 loss: 1.403679490 prediction: yf you watt you
0014 loss: 1.372415900 prediction: yf you watt you
0015 loss: 1.334304094 prediction: yf you waot you
0016 loss: 1.310002089 prediction: yf you want you
0017 loss: 1.290207148 prediction: yf you want you
0018 loss: 1.279789209 prediction: yf you watt you
0019 loss: 1.265262127 predicti

## Source - Long Sentense

In [1]:
import tensorflow as tf
import numpy as np

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
char_set = list(set(sentence)) # 유니크한 문자열을 리스트로 넣는다.
char_dic = {w: i for i, w in enumerate(char_set)} # 리스트 문자열을 index로 매칭되는 딕셔너리를 만든다.


# hyper parameters
data_dim = len(char_set)
rnn_hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10 # Any value

x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1:i+sequence_length+1]
    # print( '{0:03d} {1}->{2}'.format(i, x_str, y_str) )
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    x_data.append(x)
    y_data.append(y)
          

batch_size = len(x_data)
        
# x_data = [sample_idx[:-1]] # X data sample (0 ~ n-1) hello:hell
# y_data = [sample_idx[1:]]  # Y label sample ( 1~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

X_one_hot = tf.one_hot(X, num_classes)

cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size,  tf.float32)
outputs, _state = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)

loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

sess =  tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    loss_, _ = sess.run([loss, train], feed_dict={X:x_data, Y:y_data})

    print(i, "loss: ", loss_)

result = sess.run(prediction, feed_dict={X:x_data})

answer_full = ""
result_full = ""
i = 0
for (y, r) in zip(y_data, result):

    # print("prediction: ", r, "true Y: ", y)
    # print char using dic
    result_str = [char_set[c] for c in np.squeeze(r)]
    print("\tPrediction str: ", ''.join(result_str))

    answer_str = [char_set[c] for c in np.squeeze(y)]
    print("\tAnswer str: ", ''.join(answer_str))
    print("\tAnswer str2: ",  answer_str)

    if i==0:
        answer_full = "{}{}".format(answer_full, ''.join(answer_str))
    else:
        answer_full = "{}{}".format(answer_full, answer_str[-1])

    if i==0:
        result_full = "{}{}".format(result_full, ''.join(result_str))
    else:
        result_full = "{}{}".format(result_full, result_str[-1])

    i = i+1

print("Answer_full: ", answer_full)
print("Result_full: ", result_full)

0 loss:  3.21683
1 loss:  3.05312
2 loss:  2.94064
3 loss:  2.87805
4 loss:  2.86692
5 loss:  2.83854
6 loss:  2.82628
7 loss:  2.79839
8 loss:  2.79745
9 loss:  2.75693
10 loss:  2.71631
11 loss:  2.70143
12 loss:  2.67269
13 loss:  2.64019
14 loss:  2.61806
15 loss:  2.59375
16 loss:  2.56632
17 loss:  2.53713
18 loss:  2.51033
19 loss:  2.48948
20 loss:  2.47062
21 loss:  2.44816
22 loss:  2.43051
23 loss:  2.42116
24 loss:  2.40653
25 loss:  2.3923
26 loss:  2.38458
27 loss:  2.37335
28 loss:  2.36416
29 loss:  2.35399
30 loss:  2.34332
31 loss:  2.33718
32 loss:  2.32653
33 loss:  2.32119
34 loss:  2.31622
35 loss:  2.30851
36 loss:  2.30188
37 loss:  2.29617
38 loss:  2.2907
39 loss:  2.28622
40 loss:  2.28007
41 loss:  2.27604
42 loss:  2.27034
43 loss:  2.2658
44 loss:  2.26042
45 loss:  2.25488
46 loss:  2.2477
47 loss:  2.24124
48 loss:  2.23496
49 loss:  2.22743
50 loss:  2.22071
51 loss:  2.21305
52 loss:  2.20722
53 loss:  2.19933
54 loss:  2.19097
55 loss:  2.18548
56 los

461 loss:  1.92514
462 loss:  1.92465
463 loss:  1.92434
464 loss:  1.92408
465 loss:  1.92385
466 loss:  1.92359
467 loss:  1.92311
468 loss:  1.92296
469 loss:  1.92414
470 loss:  1.92431
471 loss:  1.92337
472 loss:  1.92336
473 loss:  1.9229
474 loss:  1.92257
475 loss:  1.92229
476 loss:  1.92193
477 loss:  1.92196
478 loss:  1.9216
479 loss:  1.92131
480 loss:  1.92124
481 loss:  1.92085
482 loss:  1.92081
483 loss:  1.92082
484 loss:  1.92196
485 loss:  1.93429
486 loss:  1.93004
487 loss:  1.93017
488 loss:  1.93411
489 loss:  1.9354
490 loss:  1.94135
491 loss:  1.93383
492 loss:  1.93469
493 loss:  1.94193
494 loss:  1.93814
495 loss:  1.94908
496 loss:  1.9395
497 loss:  1.93884
498 loss:  1.93957
499 loss:  1.93704
500 loss:  1.93177
501 loss:  1.93192
502 loss:  1.93361
503 loss:  1.93506
504 loss:  1.93529
505 loss:  1.93185
506 loss:  1.93028
507 loss:  1.92957
508 loss:  1.92888
509 loss:  1.92766
510 loss:  1.92673
511 loss:  1.92669
512 loss:  1.92682
513 loss:  1.926

903 loss:  1.94084
904 loss:  1.9417
905 loss:  1.94249
906 loss:  1.94127
907 loss:  1.94011
908 loss:  1.94047
909 loss:  1.94144
910 loss:  1.93981
911 loss:  1.93844
912 loss:  1.9377
913 loss:  1.93756
914 loss:  1.93685
915 loss:  1.93633
916 loss:  1.93593
917 loss:  1.93533
918 loss:  1.93526
919 loss:  1.93481
920 loss:  1.9343
921 loss:  1.9341
922 loss:  1.93351
923 loss:  1.93323
924 loss:  1.933
925 loss:  1.93274
926 loss:  1.93219
927 loss:  1.93198
928 loss:  1.93183
929 loss:  1.93168
930 loss:  1.93151
931 loss:  1.93125
932 loss:  1.93105
933 loss:  1.93083
934 loss:  1.93067
935 loss:  1.93051
936 loss:  1.93034
937 loss:  1.93019
938 loss:  1.93011
939 loss:  1.92994
940 loss:  1.92981
941 loss:  1.92963
942 loss:  1.92953
943 loss:  1.92944
944 loss:  1.92929
945 loss:  1.92913
946 loss:  1.92898
947 loss:  1.92882
948 loss:  1.92864
949 loss:  1.92838
950 loss:  1.92809
951 loss:  1.92801
952 loss:  1.92828
953 loss:  1.92803
954 loss:  1.92805
955 loss:  1.92826